In [1]:
import requests
import pandas as pd
from datetime import datetime
from time import sleep
from concurrent.futures import ThreadPoolExecutor, as_completed
# # ------------------- Konfiguration -------------------
# BASE_URL = "https://dataset.api.hub.geosphere.at/v1"
# DATASET = "klima-v2-1h"
# START_DATE = datetime(2014, 1, 1)
#
# # Gewünschte Parameterliste
# PARAMETERS = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]
#
# # ------------------- Schritt 1: Stationen abrufen -------------------
# def get_station_list():
#     url = f"{BASE_URL}/station/historical/{DATASET}/metadata"
#     response = requests.get(url)
#     response.raise_for_status()
#     data = response.json()              # Das ist ein Dict mit Schlüssel 'stations'
#     stations = data["stations"]         # Hier extrahieren wir die Liste
#     return [{"station_id": s["id"], "name": s["name"]} for s in stations]
#
#
# def check_availability_parallel(station_id, param):
#     url = f"{BASE_URL}/station/historical/{DATASET}/available-times"
#     params = {"station_ids": station_id, "parameters": param}
#     try:
#         response = requests.get(url, params=params, timeout=10)
#         response.raise_for_status()
#         data = response.json()
#         if param not in data or not data[param]:
#             return param, "❌"
#         start_time = data[param][0]["start"]
#         if datetime.fromisoformat(start_time) <= START_DATE:
#             return param, "✔️"
#         else:
#             return param, "❌"
#     except:
#         return param, "❌"
#
# def build_availability_table_fast():
#     stations = get_station_list()
#     availability = []
#
#     for i, station in enumerate(stations):
#         print(f"[{i+1}/{len(stations)}] Prüfe {station['name']} ({station['station_id']})...")
#         row = {"station_id": station["station_id"], "name": station["name"]}
#         with ThreadPoolExecutor(max_workers=20) as executor:
#             futures = {executor.submit(check_availability_parallel, station["station_id"], p): p for p in PARAMETERS}
#             for future in as_completed(futures):
#                 param, result = future.result()
#                 row[param] = result
#         availability.append(row)
#
#     return pd.DataFrame(availability)
#
# # ------------------- Schritt 4: Ausführen -------------------
# df = build_availability_table_fast()
#
# # Ergebnis speichern oder anzeigen
# df.to_csv("geosphere_verfuegbarkeit.csv", index=False)
# df.head()


In [2]:
#df


,station_id,name,so_h,cglo,tb10,rrm,rf,ffx,ff,tb20,tl,rr
0,1,Aflenz,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
1,2,Aigen im Ennstal,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
2,3,Allentsteig,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
3,4,Amstetten,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
4,5,Bad Aussee,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
...,...,...,...,...,...,...,...,...,...,...,...,...
806,20902,Bad Radkersburg,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
807,20903,Bad Radkersburg,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
808,21101,Loibl Tunnel,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
809,21300,Bad Eisenkappel,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌


In [7]:
station_id = 1  # Aflenz
parameter = "tl"
url = f"https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h/available-times"
params = {"station_ids": station_id, "parameters": parameter}

response = requests.get(url, params=params)
print(response.status_code)
print(response.json())

404
{'detail': 'Not Found'}


In [8]:
url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h/available-times"
params = {"station_ids": 105, "parameters": "tl"}  # Wien Hohe Warte

r = requests.get(url, params=params)
print(r.status_code)
print(r.json())

404
{'detail': 'Not Found'}


In [12]:
import requests
from datetime import datetime

# Basis-Parameter
station_id = 1
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"
start = "2014-01-01T00:00"
end = "2014-01-01T01:00"  # kurzes Zeitfenster
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]

# Ergebnisse prüfen
verfuegbarkeit = {}

for param in parameters:
    params = {
        "station_ids": station_id,
        "parameters": param,
        "start": start,
        "end": end
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        try:
            data = response.json()
            print(data)
            values = data.get("data", {}).get(str(station_id), {}).get(param, [])
            verfuegbarkeit[param] = "✔️" if any(values) else "❌"
        except Exception:
            verfuegbarkeit[param] = "❌"
    else:
        verfuegbarkeit[param] = "❌"

# Ergebnis anzeigen
print(f"Verfügbarkeit für Station {station_id} ab {start}:")
for param, status in verfuegbarkeit.items():
    print(f"{param}: {status}")

{'media_type': 'application/json', 'type': 'FeatureCollection', 'version': 'v1', 'timestamps': ['2014-01-01T00:00+00:00', '2014-01-01T01:00+00:00'], 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [47.54594, 15.24069]}, 'properties': {'parameters': {'tl': {'name': 'Lufttemperatur 2m', 'unit': '°C', 'data': [0.6, 0.8]}}, 'station': 1}}]}
{'media_type': 'application/json', 'type': 'FeatureCollection', 'version': 'v1', 'timestamps': ['2014-01-01T00:00+00:00', '2014-01-01T01:00+00:00'], 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [47.54594, 15.24069]}, 'properties': {'parameters': {'rf': {'name': 'Relative Feuchte', 'unit': '%', 'data': [96, 93]}}, 'station': 1}}]}
{'media_type': 'application/json', 'type': 'FeatureCollection', 'version': 'v1', 'timestamps': ['2014-01-01T00:00+00:00', '2014-01-01T01:00+00:00'], 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [47.54594, 15.24069]}, 'properties': {'

In [2]:
import requests

station_id = 1
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"
start = "2014-01-01T00:00"
end = "2014-01-01T01:00"
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]

verfuegbarkeit = {}

for param in parameters:
    params = {
        "station_ids": station_id,
        "parameters": param,
        "start": start,
        "end": end
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        try:
            data = response.json()
            features = data.get("features", [])
            if features:
                values = features[0]["properties"]["parameters"].get(param, {}).get("data", [])
                verfuegbarkeit[param] = "✔️" if any(v is not None for v in values) else "❌"
            else:
                verfuegbarkeit[param] = "❌"
        except Exception as e:
            print(f"Fehler bei {param}: {e}")
            verfuegbarkeit[param] = "❌"
    else:
        verfuegbarkeit[param] = "❌"

print(f"Verfügbarkeit für Station {station_id} ab {start}:")
for param, status in verfuegbarkeit.items():
    print(f"{param}: {status}")


Verfügbarkeit für Station 1 ab 2014-01-01T00:00:
tl: ✔️
rf: ✔️
ff: ✔️
ffx: ✔️
cglo: ✔️
so_h: ❌
rr: ❌
rrm: ❌
tb10: ✔️
tb20: ❌


In [3]:
import requests
import pandas as pd
import numpy as np

# Parameter
station_id = 1
start = "2014-01-01T00:00"
end = "2018-01-31T23:00"
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]
param_str = ",".join(parameters)
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"

# Anfrage
params = {
    "station_ids": station_id,
    "parameters": param_str,
    "start": start,
    "end": end
}
response = requests.get(base_url, params=params)

# Ergebnis verarbeiten
if response.status_code == 200:
    data = response.json()
    features = data.get("features", [])
    if features:
        param_data = features[0]["properties"]["parameters"]

        # Verfügbarkeit berechnen
        result = {}
        for param in parameters:
            values = param_data.get(param, {}).get("data", [])
            total = len(values)
            not_nan = np.count_nonzero([v is not None for v in values])
            percent = round(not_nan / total * 100, 2) if total > 0 else 0.0
            result[param] = percent

        # DataFrame mit einer Zeile (eine Station)
        df = pd.DataFrame([result], index=[station_id])
        df.index.name = "station_id"
        print(df)
    else:
        print("Keine Daten vorhanden.")
else:
    print(f"Fehler beim Abrufen: {response.status_code}")


               tl     rf     ff    ffx   cglo   so_h     rr    rrm  tb10  tb20
station_id                                                                    
1           99.99  99.99  99.92  99.84  99.99  100.0  99.57  99.99  8.47   0.0


In [13]:
import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed


# Konfiguration
start = "2014-01-01T00:00"
end = "2018-01-31T23:00"
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]
param_str = ",".join(parameters)
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"
max_workers = 8  # Anzahl paralleler Threads

# Beispiel: nur erste 50 Stationen zum Test
def get_station_ids(limit=None):
    url = f"{base_url}/metadata"
    response = requests.get(url)
    response.raise_for_status()
    stations = response.json()["stations"]
    return [s["id"] for s in stations[:limit]] if limit else [s["id"] for s in stations]

# Funktion für eine einzelne Station
def check_station(station_id):
    params = {
        "station_ids": station_id,
        "parameters": param_str,
        "start": start,
        "end": end
    }
    try:
        response = requests.get(base_url, params=params, timeout=60)
        if response.status_code != 200:
            print(f"Fehler bei Station {station_id}: {response.status_code}")
            return None

        data = response.json()
        features = data.get("features", [])
        if not features:
            return None

        param_data = features[0]["properties"]["parameters"]
        result = {}
        for param in parameters:
            values = param_data.get(param, {}).get("data", [])
            total = len(values)
            not_nan = np.count_nonzero([v is not None for v in values])
            percent = round(not_nan / total * 100, 2) if total > 0 else 0.0
            result[param] = percent

        return station_id, result

    except Exception as e:
        print(f"Fehler bei Station {station_id}: {e}")
        return None

# Alle Stationen parallel verarbeiten
def build_availability_dataframe(station_ids):
    rows = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(check_station, sid): sid for sid in station_ids}
        for future in as_completed(futures):
            result = future.result()
            if result:
                station_id, param_result = result
                row = pd.Series(param_result, name=station_id)
                rows.append(row)
    df = pd.DataFrame(rows)
    df.index.name = "station_id"
    return df

# Ausführen
station_ids = get_station_ids(limit=50)  # Zum Start: nur 50 Stationen
df = build_availability_dataframe(station_ids)
df.to_csv("verfuegbarkeit_geosphere_500.csv")
print(df.head())


Fehler bei Station 5: 429
Fehler bei Station 8: 429
Fehler bei Station 6: 429
Fehler bei Station 7: 429
Fehler bei Station 10: 429
Fehler bei Station 9: 429
Fehler bei Station 12: 429
Fehler bei Station 11: 429
Fehler bei Station 14: 429
Fehler bei Station 13: 429
Fehler bei Station 15: 429
Fehler bei Station 16: 429
Fehler bei Station 17: 429
Fehler bei Station 18: 429
Fehler bei Station 20: 429
Fehler bei Station 19: 429
Fehler bei Station 21: 429Fehler bei Station 22: 429

Fehler bei Station 24: 429
Fehler bei Station 23: 429
Fehler bei Station 25: 429
Fehler bei Station 26: 429
Fehler bei Station 28: 429
Fehler bei Station 27: 429
Fehler bei Station 30: 429
Fehler bei Station 29: 429
Fehler bei Station 31: 429
Fehler bei Station 32: 429
Fehler bei Station 34: 429
Fehler bei Station 33: 429
Fehler bei Station 36: 429
Fehler bei Station 37: 429
Fehler bei Station 39: 429
Fehler bei Station 38: 429
Fehler bei Station 40: 429
Fehler bei Station 42: 429
Fehler bei Station 43: 429
Fehler

In [14]:
df

,tl,rf,ff,ffx,cglo,so_h,rr,rrm,tb10,tb20
station_id,,,,,,,,,,
45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,99.97,99.97,99.85,99.86,99.99,99.99,99.67,99.97,9.45,24.29
3,99.98,99.99,99.99,99.99,99.99,100.00,99.97,99.99,100.00,99.04
2,99.94,99.78,100.00,100.00,99.93,100.00,99.82,100.00,99.92,99.94
1,99.99,99.99,99.92,99.84,99.99,100.00,99.57,99.99,8.47,0.00
46,99.94,99.95,99.94,99.96,99.95,100.00,99.98,99.99,99.96,99.96
48,100.00,100.00,100.00,100.00,99.98,100.00,100.00,100.00,99.99,99.99
47,99.96,99.96,99.47,99.47,99.95,99.95,99.85,99.95,95.13,99.47
50,99.94,99.94,100.00,100.00,100.00,100.00,99.67,99.87,0.00,0.00


In [9]:
len(df.index)

42

In [16]:
import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import time


# Konfiguration
start = "2014-01-01T00:00"
end = "2018-01-31T23:00"
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]
param_str = ",".join(parameters)
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"
max_workers = 3  # Anzahl paralleler Threads

# Beispiel: nur erste 50 Stationen zum Test
def get_station_ids(limit=None):
    url = f"{base_url}/metadata"
    response = requests.get(url)
    response.raise_for_status()
    stations = response.json()["stations"]
    return [s["id"] for s in stations[:limit]] if limit else [s["id"] for s in stations]

# Funktion für eine einzelne Station
def check_station(station_id, retries=3, delay=65):
    params = {
        "station_ids": station_id,
        "parameters": param_str,
        "start": start,
        "end": end
    }

    for attempt in range(retries):
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            try:
                data = response.json()
                features = data.get("features", [])
                if not features:
                    return None
                param_data = features[0]["properties"]["parameters"]
                result = {}
                for param in parameters:
                    values = param_data.get(param, {}).get("data", [])
                    total = len(values)
                    not_nan = np.count_nonzero([v is not None for v in values])
                    percent = round(not_nan / total * 100, 2) if total > 0 else 0.0
                    result[param] = percent
                return station_id, result
            except Exception as e:
                print(f"Fehler beim Parsen von Station {station_id}: {e}")
                return None

        elif response.status_code == 429:
            print(f"429 bei Station {station_id}, warte {delay}s...")
            time.sleep(delay)
        else:
            print(f"Fehler bei Station {station_id}: {response.status_code}")
            return None
    return None

# Alle Stationen parallel verarbeiten
def build_availability_dataframe(station_ids):
    rows = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(check_station, sid): sid for sid in station_ids}
        for future in as_completed(futures):
            result = future.result()
            if result:
                station_id, param_result = result
                row = pd.Series(param_result, name=station_id)
                rows.append(row)
    df = pd.DataFrame(rows)
    df.index.name = "station_id"
    return df

# Ausführen
station_ids = get_station_ids(limit=50)  # Zum Start: nur 50 Stationen
df = build_availability_dataframe(station_ids)
df.to_csv("../data/verfuegbarkeit_geosphere_500.csv")
print(df.head())

               tl     rf      ff     ffx   cglo    so_h     rr     rrm  \
station_id                                                               
1           99.99  99.99   99.92   99.84  99.99  100.00  99.57   99.99   
3           99.98  99.99   99.99   99.99  99.99  100.00  99.97   99.99   
2           99.94  99.78  100.00  100.00  99.93  100.00  99.82  100.00   
4           99.97  99.97   99.85   99.86  99.99   99.99  99.67   99.97   
5           99.93  99.93   99.93   99.93   0.00  100.00  98.95   99.92   

              tb10   tb20  
station_id                 
1             8.47   0.00  
3           100.00  99.04  
2            99.92  99.94  
4             9.45  24.29  
5             0.00   0.00  


In [17]:
df

,tl,rf,ff,ffx,cglo,so_h,rr,rrm,tb10,tb20
station_id,,,,,,,,,,
1,99.99,99.99,99.92,99.84,99.99,100.00,99.57,99.99,8.47,0.00
3,99.98,99.99,99.99,99.99,99.99,100.00,99.97,99.99,100.00,99.04
2,99.94,99.78,100.00,100.00,99.93,100.00,99.82,100.00,99.92,99.94
4,99.97,99.97,99.85,99.86,99.99,99.99,99.67,99.97,9.45,24.29
5,99.93,99.93,99.93,99.93,0.00,100.00,98.95,99.92,0.00,0.00
6,99.28,99.28,99.06,99.09,99.27,99.62,99.40,99.46,0.00,0.00
7,100.00,100.00,99.96,99.96,99.93,100.00,98.82,99.91,100.00,93.69
8,99.99,99.99,99.98,99.98,99.95,100.00,99.98,99.98,0.00,0.00
9,99.89,99.89,99.85,99.86,99.85,99.90,99.37,99.86,5.45,17.97


In [25]:
import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# Konfiguration
start = "2014-01-01T00:00"
end = "2018-01-31T23:00"
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]
param_str = ",".join(parameters)
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"
max_workers = 3
retry_delay = 60

# Stationen abrufen
def get_station_ids(limit=None):
    url = f"{base_url}/metadata"
    response = requests.get(url)
    response.raise_for_status()
    stations = response.json()["stations"]
    return [s["id"] for s in stations[:limit]] if limit else [s["id"] for s in stations]

# Einzelstation prüfen
def check_station(station_id, retries=3, delay=retry_delay):
    params = {
        "station_ids": station_id,
        "parameters": param_str,
        "start": start,
        "end": end
    }

    for attempt in range(retries):
        try:
            response = requests.get(base_url, params=params, timeout=120)
            if response.status_code == 200:
                data = response.json()
                features = data.get("features", [])
                if not features:
                    return station_id, None, "no_features"
                param_data = features[0]["properties"]["parameters"]
                result = {}
                for param in parameters:
                    values = param_data.get(param, {}).get("data", [])
                    total = len(values)
                    not_nan = np.count_nonzero([v is not None for v in values])
                    percent = round(not_nan / total * 100, 2) if total > 0 else 0.0
                    result[param] = percent
                return station_id, result, None
            elif response.status_code == 429:
                print(f"429 bei Station {station_id}, warte {delay}s...")
                time.sleep(delay)
            else:
                return station_id, None, f"http_{response.status_code}"
        except Exception as e:
            return station_id, None, str(e)
    return station_id, None, "max_retries"

# Parallele Verarbeitung
def build_availability_dataframe(station_ids):
    rows = []
    errors = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(check_station, sid): sid for sid in station_ids}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Verarbeite Stationen"):
            station_id, result, error = future.result()
            if result:
                row = pd.Series(result, name=station_id)
                rows.append(row)
            elif error:
                errors.append({"station_id": station_id, "error": error})

    df = pd.DataFrame(rows)
    df.index.name = "station_id"
    errors_df = pd.DataFrame(errors)
    return df, errors_df

# Ausführen
station_ids = get_station_ids(limit=None)  # anpassen für mehr Stationen
df, errors_df = build_availability_dataframe(station_ids)

# Ergebnisse speichern
df.to_csv("../data/verfuegbarkeit_geosphere.csv")
errors_df.to_csv("../data/verfuegbarkeit_geosphere_errors.csv", index=False)

print("Fertig! Ergebnisse gespeichert als:")
print("- verfügbarkeit_geosphere.csv")
print("- verfügbarkeit_geosphere_errors.csv")


Verarbeite Stationen:  28%|██▊       | 142/500 [20:50<21:59,  3.68s/it]  

429 bei Station 153, warte 60s...


Verarbeite Stationen:  29%|██▊       | 143/500 [20:58<28:44,  4.83s/it]

429 bei Station 154, warte 60s...


Verarbeite Stationen:  29%|██▉       | 144/500 [21:08<37:28,  6.32s/it]

...

429 bei Station 7890, warte 60s...


Verarbeite Stationen:  85%|████████▌ | 426/500 [1:35:51<53:37, 43.49s/it]  

429 bei Station 7900, warte 60s...


Verarbeite Stationen: 100%|██████████| 500/500 [1:45:38<00:00, 12.68s/it]

Fertig! Ergebnisse gespeichert als:
- verfügbarkeit_geosphere.csv
- verfügbarkeit_geosphere_errors.csv


In [26]:
df

,tl,rf,ff,ffx,cglo,so_h,rr,rrm,tb10,tb20
station_id,,,,,,,,,,
1,99.99,99.99,99.92,99.84,99.99,100.00,99.57,99.99,8.47,0.00
3,99.98,99.99,99.99,99.99,99.99,100.00,99.97,99.99,100.00,99.04
2,99.94,99.78,100.00,100.00,99.93,100.00,99.82,100.00,99.92,99.94
6,99.28,99.28,99.06,99.09,99.27,99.62,99.40,99.46,0.00,0.00
5,99.93,99.93,99.93,99.93,0.00,100.00,98.95,99.92,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
10510,100.00,100.00,99.99,100.00,99.99,100.00,99.91,100.00,99.99,99.99
10511,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10531,99.99,99.99,99.98,99.99,99.99,100.00,99.97,99.97,97.00,95.95


In [27]:
errors_df

,station_id,error
0,32,HTTPSConnectionPool(host='dataset.api.hub.geos...
1,34,HTTPSConnectionPool(host='dataset.api.hub.geos...
2,49,HTTPSConnectionPool(host='dataset.api.hub.geos...
3,84,HTTPSConnectionPool(host='dataset.api.hub.geos...
4,89,HTTPSConnectionPool(host='dataset.api.hub.geos...
...,...,...
61,7906,HTTPSConnectionPool(host='dataset.api.hub.geos...
62,8805,HTTPSConnectionPool(host='dataset.api.hub.geos...
63,9605,HTTPSConnectionPool(host='dataset.api.hub.geos...
64,10111,HTTPSConnectionPool(host='dataset.api.hub.geos...


In [28]:
import requests
import pandas as pd

# Konfiguration
station_id = 18
start = "2014-01-01T00:00"
end = "2018-01-31T23:00"
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]
param_str = ",".join(parameters)
url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"

# API-Abfrage
params = {
    "station_ids": station_id,
    "parameters": param_str,
    "start": start,
    "end": end
}

response = requests.get(url, params=params)
data = response.json()

# Daten extrahieren
features = data.get("features", [])
if not features:
    print("Keine Daten gefunden.")
else:
    timestamps = data.get("timestamps", [])
    param_data = features[0]["properties"]["parameters"]

    # DataFrame mit allen Parametern aufbauen
    df = pd.DataFrame(index=pd.to_datetime(timestamps))
    for param in parameters:
        values = param_data.get(param, {}).get("data", [])
        df[param] = values

    df.index.name = "timestamp"
    print(df.head())


                            tl    rf   ff   ffx  cglo  so_h   rr  rrm  tb10  \
timestamp                                                                     
2014-01-01 00:00:00+00:00  3.6  92.0  4.6   8.9   0.0   0.0  0.0  0.0   4.4   
2014-01-01 01:00:00+00:00  3.8  92.0  4.9   8.3   0.0   0.0  0.0  0.0   4.4   
2014-01-01 02:00:00+00:00  3.9  91.0  5.1   9.4   0.0   0.0  0.0  0.0   4.4   
2014-01-01 03:00:00+00:00  3.9  90.0  5.3  10.8   0.0   0.0  0.0  0.0   4.4   
2014-01-01 04:00:00+00:00  3.8  90.0  4.9   9.6   0.0   0.0  0.0  0.0   4.4   

                           tb20  
timestamp                        
2014-01-01 00:00:00+00:00   5.0  
2014-01-01 01:00:00+00:00   4.9  
2014-01-01 02:00:00+00:00   4.9  
2014-01-01 03:00:00+00:00   4.9  
2014-01-01 04:00:00+00:00   4.9  


In [30]:
df.describe()

,tl,rf,ff,ffx,cglo,so_h,rr,rrm,tb10,tb20
count,35787.000000,35806.000000,35805.000000,35805.000000,35803.000000,35808.000000,35765.000000,35803.000000,35681.000000,35806.000000
mean,11.343706,75.083869,2.869326,6.569728,136.268078,0.243856,0.066344,4.762199,11.578952,12.242591
std,8.605634,17.179296,1.956521,3.532581,218.289399,0.394665,0.495662,14.256427,7.663214,7.575967
min,-13.800000,16.000000,0.000000,0.600000,0.000000,0.000000,0.000000,0.000000,-3.600000,-1.900000
25%,4.600000,64.000000,1.400000,3.800000,0.000000,0.000000,0.000000,0.000000,4.700000,5.300000
50%,11.000000,78.000000,2.500000,6.000000,5.000000,0.000000,0.000000,0.000000,11.000000,11.600000
75%,17.600000,89.000000,3.900000,8.700000,195.000000,0.500000,0.000000,0.000000,18.600000,19.500000
max,36.700000,100.000000,16.600000,34.900000,968.000000,1.000000,26.300000,60.000000,28.800000,27.600000


In [32]:
import requests
import pandas as pd
import numpy as np
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Konfiguration
start = "2014-01-01T00:00"
end = "2018-01-31T23:00"
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]
param_str = ",".join(parameters)
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"
max_workers = 1
retry_delay = 60
checkpoint_path = "../data/checkpoint.csv"
error_log_path = "../data/errors.csv"

# Lade bereits verarbeitete Stationen
def load_processed_ids():
    if os.path.exists(checkpoint_path):
        df = pd.read_csv(checkpoint_path, index_col="station_id")
        return set(df.index), df
    else:
        return set(), pd.DataFrame()

# Alle Stationen holen
def get_station_ids():
    url = f"{base_url}/metadata"
    response = requests.get(url)
    response.raise_for_status()
    stations = response.json()["stations"]
    return [s["id"] for s in stations]

# Einzelergebnis prüfen
def check_station(station_id, retries=3, delay=retry_delay):
    params = {
        "station_ids": station_id,
        "parameters": param_str,
        "start": start,
        "end": end
    }
    for attempt in range(retries):
        try:
            response = requests.get(base_url, params=params, timeout=90)
            if response.status_code == 200:
                data = response.json()
                features = data.get("features", [])
                if not features:
                    return station_id, None, "no_features"
                param_data = features[0]["properties"]["parameters"]
                result = {}
                for param in parameters:
                    values = param_data.get(param, {}).get("data", [])
                    total = len(values)
                    not_nan = np.count_nonzero([v is not None for v in values])
                    percent = round(not_nan / total * 100, 2) if total > 0 else 0.0
                    result[param] = percent
                return station_id, result, None
            elif response.status_code == 429:
                time.sleep(delay * (attempt + 1))
            else:
                return station_id, None, f"http_{response.status_code}"
        except Exception as e:
            return station_id, None, str(e)
    return station_id, None, "max_retries"

# Hauptlogik mit Zwischenspeicherung
def process_all_stations():
    all_ids = get_station_ids()
    processed_ids, df_checkpoint = load_processed_ids()
    to_process = [sid for sid in all_ids if sid not in processed_ids]
    errors = []

    print(f"Starte mit {len(to_process)} verbleibenden Stationen...")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(check_station, sid): sid for sid in to_process}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Stationen"):
            station_id, result, error = future.result()
            if result:
                row = pd.Series(result, name=station_id)
                df_checkpoint = pd.concat([df_checkpoint, row.to_frame().T])
                df_checkpoint.index.name = "station_id"
                df_checkpoint.to_csv(checkpoint_path)  # regelmäßig speichern
            elif error:
                errors.append({"station_id": station_id, "error": error})
                pd.DataFrame(errors).to_csv(error_log_path, index=False)

    print("Fertig! Ergebnisse gespeichert in:")
    print(f" - {checkpoint_path}")
    print(f" - {error_log_path} (Fehlerlog)")

# Script starten
if __name__ == "__main__":
    process_all_stations()


Starte mit 289 verbleibenden Stationen...


Stationen: 100%|██████████| 289/289 [45:27<00:00,  9.44s/it]   

Fertig! Ergebnisse gespeichert in:
 - ../data/checkpoint.csv
 - ../data/errors.csv (Fehlerlog)
